In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from datetime import datetime


In [2]:
driver = webdriver.Chrome()
driver.get("https://www.nuerburgring-langstrecken-serie.de/en/calendar-nurburgring-langstrecken-serie-2024/")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('tr')

driver.quit()


In [3]:
# schedule_details

In [4]:
days = [] 
months = []
titles = [] 
descs = []
circuit = 'Nurburgring Nordschleife'
time = []
start_hours = []
start_minutes = []
end_hours = []
end_minutes = []

year = datetime.now().year

round_counter = 1


month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'Dec': 12,
        'DEC': 12
    }



In [5]:
##Example building info lists

# for i in schedule_details:
#     temp_circuit = i.find('div', {'class': 'race-circuit'}).text
#     circuits.append(temp_circuit)
    
#     temp_date = i.find('div', {'class':"race-date"}).text
#     temp_date = temp_date.split(' ')
#     days.append(int(temp_date[0]))
    
#     temp_month = temp_date[1]
#     converted_month = month_mapping[temp_month]
#     months.append(converted_month)
    
#     round_counter = 1
#     temp_title = 'AsLMS Round' + ' ' + str(round_counter)
#     round_counter += 1
#     titles.append(temp_title)

In [6]:


for i in schedule_details:
    temp_date = i.find_all('td')[0].text.split('.')
  
    
    if '+' in temp_date[1]:  # Handle instances with a '+'
        first_day = int(temp_date[0])
        last_day = int(temp_date[1].split('+')[1])
        month = int(temp_date[2])
        days.append(first_day)
        days.append(last_day)
        months.append(month)
        months.append(month) #This line is repeated to ensure there is a matching number of months to days
        temp_title = 'N24 Qualifying Round' + ' ' + str(round_counter)
        round_counter += 1
        titles.append(temp_title)
        temp_title = 'N24 Qualifying Round' + ' ' + str(round_counter)
        round_counter += 1
        titles.append(temp_title)
    
        desc = i.find_all('td')[1].text
        descs.append(desc)
        descs.append(desc)
        
        start_hours.append(11)
        start_minutes.append(0)
        
        start_hours.append(11)
        start_minutes.append(0)
    else:
        day = int(temp_date[0])
        month = int(temp_date[1])
        days.append(day)
        months.append(month)
        
        temp_title = 'NLS Round' + ' ' + str(round_counter)
        round_counter += 1
        titles.append(temp_title)

        desc = i.find_all('td')[1].text
        descs.append(desc)
        
        start_hour = 11
        start_hours.append(start_hour)
        start_minutes.append(0)
        
for i in descs:
    if '6h' in i:
        end_hour = start_hour + 6
        end_hours.append(end_hour)
    else:
        end_hour = start_hour + 4
        end_hours.append(end_hour)

    

In [7]:
descs

['NLS1: 64. ADAC ACAS Cup',
 'NLS2: 63. ADAC Reinoldus-Langstreckenrennen',
 'ADAC 24h Nürburgring Qualifiers',
 'ADAC 24h Nürburgring Qualifiers',
 'NLS3: 69. ADAC Westfalenfahrt',
 'NLS4: 6h ADAC Ruhr-Pokal-Rennen',
 'NLS5: 55. Adenauer ADAC Rundstrecken-Trophy',
 'NLS6: 56. ADAC Barbarossapreis']

In [8]:
start_hours

[11, 11, 11, 11, 11, 11, 11, 11]

In [9]:
end_hours

[15, 15, 15, 15, 15, 17, 15, 15]

In [10]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [11]:
NLS = Calendar()
NLS.add('prodid', '-//My calendar product//example.com//')
NLS.add('version', '2.0')
uid = 110

In [12]:
 
for title, desc, day, month, s_h, e_h in zip(titles, descs, days, months, start_hours, end_hours):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(year, month, day, s_h, 0, 0, tzinfo=pytz.timezone('Europe/Paris')))
    ievent.add('dtend', datetime(year, month, day, e_h, 0, 0, tzinfo=pytz.timezone('Europe/Paris')))
    ievent.add('dtstamp', datetime(year, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    NLS.add_component(ievent)

In [13]:
NLS

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'NLS Round 1''), 'DESCRIPTION': vText('b'NLS1: 64. ADAC ACAS Cup''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000002302F723D30>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000002302F723DC0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000002302F723EE0>, 'LOCATION': vText('b'Nurburgring Nordschleife''), 'UID': vText('b'110'')}), VEVENT({'SUMMARY': vText('b'NLS Round 2''), 'DESCRIPTION': vText('b'NLS2: 63. ADAC Reinoldus-Langstreckenrennen''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000002302F7270D0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000002302F727160>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000002302F7271F0>, 'LOCATION': vText('b'Nurburgring Nordschleife''), 'UID': vText('b'111'')}), VEVENT({'SUMMARY': vText('b'N24 Qualifying Round 3''), 'DESCRIPTION': vText('b'ADAC 24h N\xc3\xbcrburgring Qualifiers''), 'DTSTART'

In [14]:
br

NameError: name 'br' is not defined

In [15]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'NLS.ics'), 'wb')
f.write(NLS.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
